In [ ]:
pip install speechbrain

In [ ]:
#Conservative spectral Gatting

# Re-importing necessary libraries and redefining the functions
import librosa
import numpy as np
from scipy.signal import stft, istft
from scipy.io.wavfile import write


# Redefining the noise reduction and normalization functions
def conservative_spectral_gating_noise_reduction(signal, noise_segment, n_fft=2048, hop_length=512, alpha=0.3):
    _, _, Zxx = stft(signal, nperseg=n_fft, noverlap=hop_length)
    _, _, Zxx_noise = stft(noise_segment, nperseg=n_fft, noverlap=hop_length)
    noise_mean = np.mean(np.abs(Zxx_noise), axis=1)
    gain = np.maximum(1 - alpha * (noise_mean[:, np.newaxis] / (np.abs(Zxx) + 1e-10)), 0)
    Zxx_reduced = gain * Zxx
    _, reduced_signal = istft(Zxx_reduced, noverlap=hop_length)
    reduced_signal *= np.linalg.norm(signal) / np.linalg.norm(reduced_signal)
    return reduced_signal


def normalize_audio(input_signal, target_rms=0.1):
    current_rms = np.sqrt(np.mean(np.square(input_signal)))
    if current_rms == 0:
        return input_signal
    factor = target_rms / current_rms
    normalized_audio = input_signal * factor
    return normalized_audio


# Load the uploaded file again
file_path = '/content/drive/MyDrive/Category 2/Wrong Denoised Diarization/BEA63649-C453-4D44-8551-33EA9191697E.wav'
signal, sr = librosa.load(file_path, sr=None)

# Assume the first second of the audio is representative of the noise in the file.
noise_segment = signal[:sr]  # First second for noise

# Apply the conservative spectral gating noise reduction method again
conservative_spectral_reduced_audio = conservative_spectral_gating_noise_reduction(signal, noise_segment)

# Normalize the amplitude of the reduced audio again
conservative_spectral_normalized_audio = normalize_audio(conservative_spectral_reduced_audio)

# Save the processed audio to a new file again
conservative_processed_file_path = '/content/drive/MyDrive/combined/2.wav'
write(conservative_processed_file_path, sr, conservative_spectral_normalized_audio.astype(np.float32))

# Provide the path to the processed file
conservative_processed_file_path

'/content/drive/MyDrive/combined/2.wav'